# Pre-Processing - Retail Forecasting

## Pre-Work

### Import Modules and Data

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import pickle
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Change to correct directory
os.chdir('../data/interim')

In [3]:
# Turn off setting with copy warning
pd.set_option('mode.chained_assignment', None)

In [4]:
# Load Data and inspect head
df = pd.read_csv('data post-eda.csv')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Month,Week
0,1,1,2010-01-10,20094.19,0,71.89,2.603,NaN,NaN,NaN,NaN,NaN,211.671989,7.838,2,151315,1,1
1,1,2,2010-01-10,45829.02,0,71.89,2.603,NaN,NaN,NaN,NaN,NaN,211.671989,7.838,2,151315,1,1
2,1,3,2010-01-10,9775.17,0,71.89,2.603,NaN,NaN,NaN,NaN,NaN,211.671989,7.838,2,151315,1,1
3,1,4,2010-01-10,34912.45,0,71.89,2.603,NaN,NaN,NaN,NaN,NaN,211.671989,7.838,2,151315,1,1
4,1,5,2010-01-10,23381.38,0,71.89,2.603,NaN,NaN,NaN,NaN,NaN,211.671989,7.838,2,151315,1,1


My approach is going to be dropping the markdown columns in order to have access to the entire data set for training.

I plan to create 3 separate models, one for each of the different store types. My hope is to limit the scope of each model to improve individual peformance.

In [5]:
# Dropping markdown columns
df.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace=True)

## Generate Dummy Features

Columns that should have dummy features encoded:
- Store
- Dept
- Type
- Month
- Week

Store dummy features will be generated after splitting the data by store type. This will reduce the number of unnecessary features in each subset.

In [6]:
# Create dummy features
deptDummies = pd.get_dummies(df['Dept'], prefix='Dept', drop_first=True)
monthDummies = pd.get_dummies(df['Month'], prefix='Month', drop_first=True)
weekDummies = pd.get_dummies(df['Week'], prefix='Week', drop_first=True)

In [7]:
# Join dummy features onto dataframe
df = df.join([deptDummies, monthDummies, weekDummies])

In [8]:
# Drop original features
df.drop(columns=['Dept', 'Month', 'Week'], inplace=True)

In [9]:
# Split the data by 3 store types
df0 = df.loc[df.Type == 0]
df1 = df.loc[df.Type == 1]
df2 = df.loc[df.Type == 2]

In [10]:
# Drop Type and Date features
df0 = df0.drop(columns=['Type'])
df1 = df1.drop(columns=['Type'])
df2 = df2.drop(columns=['Type'])

In [11]:
# Create Store Dummy Features
storeDummies0 = pd.get_dummies(df0['Store'], prefix='Store', drop_first=True)
storeDummies1 = pd.get_dummies(df1['Store'], prefix='Store', drop_first=True)
storeDummies2 = pd.get_dummies(df2['Store'], prefix='Store', drop_first=True)

In [12]:
# Join Store dummes to dataframes
df0 = df0.join(storeDummies0)
df1 = df1.join(storeDummies1)
df2 = df2.join(storeDummies2)

In [13]:
# Drop Store columns from dataframes
df0 = df0.drop(columns=['Store'])
df1 = df1.drop(columns=['Store'])
df2 = df2.drop(columns=['Store'])

In [14]:
# Print dataframe shapes
print(df0.shape)
print(df1.shape)
print(df2.shape)

(42597, 155)
(163495, 166)
(215478, 171)


## Train Test Split

We want to perform a train test split prior to scaling the data

In [15]:
# Create masks
trainmask0 = df0.Date < '2012-01-01'
testmask0 = df0.Date >= '2012-01-01'
trainmask1 = df1.Date < '2012-01-01'
testmask1 = df1.Date >= '2012-01-01'
trainmask2 = df2.Date < '2012-01-01'
testmask2 = df2.Date >= '2012-01-01'

# Split into sets by masks
dftrain0 = df0.loc[trainmask0]
dftest0 = df0.loc[testmask0]
dftrain1 = df1.loc[trainmask1]
dftest1 = df1.loc[testmask1]
dftrain2 = df2.loc[trainmask2]
dftest2 = df2.loc[testmask2]

## Data Scaling

Scaler transformation for continuous feature variables

In [16]:
# Initialize scaler and fit_transform X_train
scaler0 = MinMaxScaler()
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()

# List of features to transform
flist = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']

# Fit and transform training data
dftrain0[flist] = scaler0.fit_transform(dftrain0[flist])
dftrain1[flist] = scaler1.fit_transform(dftrain1[flist])
dftrain2[flist] = scaler2.fit_transform(dftrain2[flist])

In [17]:
# Verify that tranformation was successful
dftrain0.head()

,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Dept_2,Dept_3,...,Week_48,Week_49,Week_50,Week_51,Week_52,Store_37,Store_38,Store_42,Store_43,Store_44
1993,2010-01-10,9843.45,0,0.648274,0.052725,0.915722,0.253188,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1994,2010-01-10,13898.10,0,0.648274,0.052725,0.915722,0.253188,1.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1995,2010-01-10,755.67,0,0.648274,0.052725,0.915722,0.253188,1.0,0,1,...,0,0,0,0,0,0,0,0,0,0
1996,2010-01-10,13610.70,0,0.648274,0.052725,0.915722,0.253188,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,2010-01-10,444.76,0,0.648274,0.052725,0.915722,0.253188,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Apply scaler transformation to test data
dftest0[flist] = scaler0.transform(dftest0[flist])
dftest1[flist] = scaler1.transform(dftest1[flist])
dftest2[flist] = scaler2.transform(dftest2[flist])

## X y split

In [19]:
# Split X & y

# Store type 0
X_train0 = dftrain0.drop(columns=['Date', 'Weekly_Sales'])
y_train0 = dftrain0[['Weekly_Sales']]
X_test0 = dftest0.drop(columns=['Date', 'Weekly_Sales'])
y_test0 = dftest0[['Weekly_Sales']]

# Store type 1
X_train1 = dftrain1.drop(columns=['Date', 'Weekly_Sales'])
y_train1 = dftrain1[['Weekly_Sales']]
X_test1 = dftest1.drop(columns=['Date', 'Weekly_Sales'])
y_test1 = dftest1[['Weekly_Sales']]

# Store type 2
X_train2 = dftrain2.drop(columns=['Date', 'Weekly_Sales'])
y_train2 = dftrain2[['Weekly_Sales']]
X_test2 = dftest2.drop(columns=['Date', 'Weekly_Sales'])
y_test2 = dftest2[['Weekly_Sales']]

## Time-Series Cross-Validation Split

Let's define a function for splitting the training data into segments in order to perform cross-validation on models later.

In [20]:
def cvsplit(df):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    # Create masks for 5-fold CV
    trainm1 = df.Date < '2011-01-01'
    testm1 = (df.Date >= '2011-01-01') & (df.Date < '2011-03-01')
    trainm2 = df.Date < '2011-03-01'
    testm2 = (df.Date >= '2011-03-01') & (df.Date < '2011-05-01')
    trainm3 = df.Date < '2011-05-01'
    testm3 = (df.Date >= '2011-05-01') & (df.Date < '2011-07-01')
    trainm4 = df.Date < '2011-07-01'
    testm4 = (df.Date >= '2011-07-01') & (df.Date < '2011-09-01')
    trainm5 = df.Date < '2011-09-01'
    testm5 = (df.Date >= '2011-09-01') & (df.Date < '2012-01-01')
    
    # Split into sets by masks
    dftrain1 = df.loc[trainm1]
    dftest1 = df.loc[testm1]    
    dftrain2 = df.loc[trainm2]
    dftest2 = df.loc[testm2]   
    dftrain3 = df.loc[trainm3]
    dftest3 = df.loc[testm3]   
    dftrain4 = df.loc[trainm4]
    dftest4 = df.loc[testm4]   
    dftrain5 = df.loc[trainm5]
    dftest5 = df.loc[testm5]
    
    # Test train split and append to lists
    X_train.append(dftrain1.drop(columns=['Date', 'Weekly_Sales']))
    y_train.append(dftrain1[['Weekly_Sales']])
    X_test.append(dftest1.drop(columns=['Date', 'Weekly_Sales']))
    y_test.append(dftest1[['Weekly_Sales']])
    
    X_train.append(dftrain2.drop(columns=['Date', 'Weekly_Sales']))
    y_train.append(dftrain2[['Weekly_Sales']])
    X_test.append(dftest2.drop(columns=['Date', 'Weekly_Sales']))
    y_test.append(dftest2[['Weekly_Sales']])
    
    X_train.append(dftrain3.drop(columns=['Date', 'Weekly_Sales']))
    y_train.append(dftrain3[['Weekly_Sales']])
    X_test.append(dftest3.drop(columns=['Date', 'Weekly_Sales']))
    y_test.append(dftest3[['Weekly_Sales']])
    
    X_train.append(dftrain4.drop(columns=['Date', 'Weekly_Sales']))
    y_train.append(dftrain4[['Weekly_Sales']])
    X_test.append(dftest4.drop(columns=['Date', 'Weekly_Sales']))
    y_test.append(dftest4[['Weekly_Sales']])
    
    X_train.append(dftrain5.drop(columns=['Date', 'Weekly_Sales']))
    y_train.append(dftrain5[['Weekly_Sales']])
    X_test.append(dftest5.drop(columns=['Date', 'Weekly_Sales']))
    y_test.append(dftest5[['Weekly_Sales']])
    
    # Return CV test train lists
    return X_train, y_train, X_test, y_test

In [21]:
# Create cross-validation sets
cvX_train0, cvy_train0, cvX_test0, cvy_test0 = cvsplit(dftrain0)
cvX_train1, cvy_train1, cvX_test1, cvy_test1 = cvsplit(dftrain1)
cvX_train2, cvy_train2, cvX_test2, cvy_test2 = cvsplit(dftrain2)

## Export Pre-Processed Data

In [22]:
os.chdir('../processed')

In [23]:
X_train0.to_csv('X_train0.csv', index=False)
y_train0.to_csv('y_train0.csv', index=False)
X_test0.to_csv('X_test0.csv', index=False)
y_test0.to_csv('y_test0.csv', index=False)
pickle.dump(cvX_train0, open("cvX_train0", "wb"))
pickle.dump(cvy_train0, open("cvy_train0", "wb"))
pickle.dump(cvX_test0, open("cvX_test0", "wb"))
pickle.dump(cvy_test0, open ("cvy_test0", "wb"))

X_train1.to_csv('X_train1.csv', index=False)
y_train1.to_csv('y_train1.csv', index=False)
X_test1.to_csv('X_test1.csv', index=False)
y_test1.to_csv('y_test1.csv', index=False)
pickle.dump(cvX_train1, open("cvX_train1", "wb"))
pickle.dump(cvy_train1, open("cvy_train1", "wb"))
pickle.dump(cvX_test1, open("cvX_test1", "wb"))
pickle.dump(cvy_test1, open("cvy_test1", "wb"))

X_train2.to_csv('X_train2.csv', index=False)
y_train2.to_csv('y_train2.csv', index=False)
X_test2.to_csv('X_test2.csv', index=False)
y_test2.to_csv('y_test2.csv', index=False)
pickle.dump(cvX_train2, open("cvX_train2", "wb"))
pickle.dump(cvy_train2, open("cvy_train2", "wb"))
pickle.dump(cvX_test2, open("cvX_test2", "wb"))
pickle.dump(cvy_test2, open("cvy_test2", "wb"))